In [ ]:
from constants import DatasetPath

import pandas as pd
import numpy as np
import torch
import timm

from torch import nn

from skimage import io

from tqdm.notebook import tqdm

In [ ]:
# Main Dataset folder path
DATA_DIR = DatasetPath.effectivePath

# Device to run calculations on 
DEVICE = 'cuda'

In [ ]:
# Read '.csv' file containing 'anchor', 'positive' and 'negative' triplets
df = pd.read_csv(DATA_DIR + "input.csv")

In [ ]:
class APN_Model(nn.Module):
	"""
	Defines a neural network model class APN_Model that uses an EfficientNet (specifically the B0 version) as its backbone.
	"""

	def __init__(self, emb_size = 512):
		"""
		Initializes the APN_Model with a specific model and a classifier that outputs embedding vector of the specified size.

		Parameters
		----------
		emb_size (int, optional): the size of the output embedding vector (default is 512).
		"""
		super(APN_Model, self).__init__()

		# Define the model to use 
		self.efficientnet = timm.create_model('tf_efficientnetv2_b0', pretrained = True)
		
		# Replace the classifier layer with a linear layer that outputs embeddings of size `emb_size`
		self.efficientnet.classifier = nn.Linear(in_features=self.efficientnet.classifier.in_features, out_features = emb_size)

	def forward(self, images):
		"""
		Performs the forward pass of the model, which takes a batch of images and returns their embeddings.

		Parameters
		----------
			images (torch.Tensor): a batch of images to process.

		Returns
		-------
			embeddings (torch.Tensor): a batch of embeddings of size `emb_size`.
		"""
		embeddings = self.efficientnet(images)
		return embeddings

In [ ]:
# NN mod to accept greyscale fourier spectrum images instead of RGB

model = APN_Model()
model.efficientnet.conv_stem = nn.Conv2d(1, 32, 3, 2, 1, bias=False)

model.to(DEVICE)

In [ ]:
def getImageEmbeddings(img_path, model):
	"""
	Generates embeddings for a given image using the provided model.

	Parameters
	----------
		img_path (str): the path to the input image.
		model (torch.nn.Module): the PyTorch model used to generate the image embeddings.

	Returns
	-------
		img_enc (numpy.ndarray): the embeddings of the input image.
	"""
	
	# Read the image from the specified directory
	img = io.imread(DATA_DIR + img_path)

	# Add a new dimension to the image array to match the expected input shape of the model
	img = np.expand_dims(img, 0)
	
	# Convert the NumPy array to a PyTorch tensor and normalize pixel values to the range [0, 1]
	img = torch.from_numpy(img) / 255.0
	
	# Set the model to evaluation mode
	model.eval()
	
	# Disable gradient calculation for efficiency
	with torch.no_grad():
		# Move the image tensor to the appropriate device (CPU or GPU)
		img = img.to(DEVICE)
		
		# Add a batch dimension, pass the image through the model to get the embeddings
		img_enc = model(img.unsqueeze(0))
		
		# Detach the embeddings from the computation graph and move them back to the CPU
		img_enc = img_enc.detach().cpu().numpy()
		
		# Convert the embeddings to a NumPy array
		img_enc = np.array(img_enc)

	return img_enc

In [ ]:
embedding_dict = {}  # Initialize an empty dictionary to store image embeddings

# Loop through each image path in the 'anchor' column of the DataFrame
for img_path in tqdm(df['anchor'], total=len(df), desc="Generating embeddings"):
    # Generate and store the embedding for each anchor image in the dictionary
    embedding_dict[img_path] = getImageEmbeddings(img_path, model)

# Map the embeddings to the 'anchor_emb' column using the paths from the 'anchor' column
df['anchor_emb'] = df['anchor'].map(embedding_dict)

# Map the embeddings to the 'positive_emb' column using the paths from the 'positive' column
df['positive_emb'] = df['positive'].map(embedding_dict)

# Map the embeddings to the 'negative_emb' column using the paths from the 'negative' column
df['negative_emb'] = df['negative'].map(embedding_dict)

In [ ]:
def euclidean_distance(a, b):
    """
    Calculate the Euclidean distance between two vectors.

    Parameters
    ----------
    a (numpy.ndarray): the first vector.
    b (numpy.ndarray): the second vector.

    Returns
    -------
    (float): the Euclidean distance between vectors `a` and `b`.
    """
    return np.linalg.norm(a - b)  # Calculate and return the Euclidean distance

In [ ]:
# Calculate the distances between anchor and positive embeddings
df['dist_anchor_positive'] = [euclidean_distance(row['anchor_emb'], row['positive_emb']) for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Calculating distances (anchor-positive)")]

# Calculate the distances between anchor and negative embeddings
df['dist_anchor_negative'] = [euclidean_distance(row['anchor_emb'], row['negative_emb']) for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Calculating distances (anchor-negative)")]

In [ ]:
margin = 0.2  # Set the margin for semi-hard triplets

# Filter the DataFrame to select semi-hard triplets based on the defined condition
semi_hard_triplets = df[
    (df['dist_anchor_positive'] < df['dist_anchor_negative']) &  # Check if the distance between anchor and positive embeddings is less than the distance between anchor and negative embeddings
    (df['dist_anchor_negative'] < (df['dist_anchor_positive'] + margin))  # Check if the distance between anchor and negative embeddings is less than the distance between anchor and positive embeddings plus the margin
]

# Print the semi-hard triplets
print(semi_hard_triplets)

In [ ]:
def df_to_csv(df, filename, path):
	"""
	Splits the DataFrame in chunks to enable tqdm progress visualization while converting the DataFrame into a '.csv' file.

	Parametres
	----------
		df (pd.DataFrame): the DataFrame to convert.
		filename (str): the desired file name (comprehensive of '.csv' extension).
		path (str): the path where the '.csv' will be stored.
	"""
	chunks = np.array_split(df.index, 100)
	for chunck, subset in enumerate(tqdm(chunks, desc="Creating \'" + filename + "\' file")):
		if chunck == 0: # first row
			df.loc[subset].to_csv(path, mode='w', index=False)
		else:
			df.loc[subset].to_csv(path, header=None, mode='a', index=False)

	print("\'" + filename + "\' has been successfully created.")

In [ ]:
# Create a new DataFrame containing only the 'anchor', 'positive', and 'negative' columns from semi_hard_triplets
filtered_input_df = semi_hard_triplets[['anchor', 'positive', 'negative']].copy()

# Print the selected triplets DataFrame
print(filtered_input_df)

# Save to '.csv'
df_to_csv(filtered_input_df, "filtered_input.csv", DATA_DIR + "filtered_input.csv")